In [ ]:
from __future__ import print_function
from itertools import count

import torch
import torch.nn.functional as F

In [ ]:
W_target = torch.tensor([[1.]])
b_target = torch.tensor([0.])

print(W_target)
print(b_target)

In [ ]:
def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target.item()


def poly_desc(W, b):
    """Creates a string description of a polynomial."""
    result = 'y = '
    for i, w in enumerate(W):
        result += '{:+.2f} x^{} '.format(w, i + 1)
    result += '{:+.2f}'.format(b[0])
    return result


def get_batch(batch_size=32):
    """Builds a batch i.e. (x, f(x)) pair."""
    x = torch.randn(batch_size,1)
    y = f(x)
    return x, y


# Define model
torch_model = torch.nn.Linear(W_target.size(0), 1)

for batch_idx in count(1):
    # Get data
    batch_x, batch_y = get_batch()

    # Reset gradients
    torch_model.zero_grad()

    # Forward pass
    output = F.smooth_l1_loss(torch_model(batch_x), batch_y)
    loss = output.item()

    # Backward pass
    output.backward()

    # Apply gradients
    for param in torch_model.parameters():
        param.data.add_(-0.1 * param.grad)

    # Stop criterion
    if loss < 1e-3:
        break

print('Loss: {:.6f} after {} batches'.format(loss, batch_idx))
print('==> Learned function:\t' + poly_desc(torch_model.weight.view(-1), torch_model.bias))
print('==> Actual function:\t' + poly_desc(W_target.view(-1), b_target))

In [ ]:
!pip install onnxscript

In [ ]:
import onnx
import onnxscript

torch_input = torch.randn(1,1)
torch_output = torch_model(torch_input)
onnx_program = torch.onnx.export(
    torch_model,
    torch_input,
    "identity_gemm_w1x1_b1.onnx",
    export_params = True,
    input_names = ['InclusiveKinematicsElectron.x'],
    output_names = ['InclusiveKinematicsML.x'],
)

In [ ]:
import onnx
onnx_model = onnx.load("identity_gemm_w1x1_b1.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
!pip install onnxruntime

In [ ]:
import numpy as np
import onnxruntime

ort_session = onnxruntime.InferenceSession("identity_gemm_w1x1_b1.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_input = {ort_session.get_inputs()[0].name: to_numpy(torch_input)}
ort_output = ort_session.run(None, ort_input)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_output), ort_output[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")